# KPI Validation

### Download the dataset from [Kaggle](https://www.kaggle.com/carrie1/ecommerce-data) and update path accordingly.

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Union

from IPython.core.display import display

## Load/Preprocess Ecommerce data

In [2]:
ecom_df = pd.read_csv('./data/ecom_data.csv')

# A few rows have negative values for UnitPrice and Quantity. Consider only non-negative UnitPrice & Quantity
ecom_df = ecom_df[ecom_df['UnitPrice'] >= 0]
ecom_df = ecom_df[ecom_df['Quantity'] >= 0]

# Turn CustomerID into an object, instead of numerical
ecom_df['CustomerID'] = ecom_df['CustomerID'].astype(str)

# Convert InvoiceDate to a datetime object
ecom_df['date'] = pd.to_datetime(ecom_df['InvoiceDate'])
# ecom_df['InvoiceDate'] = pd.to_datetime(ecom_df['InvoiceDate'])
# ecom_df.rename(columns={'InvoiceDate': 'date'}, inplace=True)

# Feature extraction for PeriodOfDay
def get_day_period(hour: int) -> str:
    if hour < 6:
        return "night"
    elif 6 <= hour < 12:
        return "morning"
    elif 12 <= hour < 17:
        return "afternoon"
    elif 17 <= hour < 22:
        return "evening"
    else:
        return "night"
ecom_df['PeriodOfDay'] = ecom_df['date'].apply(lambda d: get_day_period(d.hour))

# Feature extraction for DayOfWeek
ecom_df['DayOfWeek'] = ecom_df['date'].apply(lambda d: d.strftime('%A'))

# Drop Description, not needed. Keep StockCode(useful for differentiating the products)
ecom_df.drop('Description', axis=1, inplace=True)

# FeatureExtraction for ItemTotalPrice
ecom_df['ItemTotalPrice'] = ecom_df['UnitPrice'] * ecom_df['Quantity']

### For testing purposes, I'll create some date columns in different forms

In [3]:
# format: %m%Y%d %X
ecom_df['date2'] = ecom_df['date'].apply(lambda d: d.strftime('%m%Y%d %X'))

# Unix time in seconds
ecom_df['unix_secs'] = ecom_df['date'].apply(lambda d: d.timestamp()).astype(int)

In [4]:
ecom_df

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,PeriodOfDay,DayOfWeek,ItemTotalPrice,date2,unix_secs
0,536365,85123A,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,2010-12-01 08:26:00,morning,Wednesday,15.30,12201001 08:26:00,1291191960
1,536365,71053,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01 08:26:00,morning,Wednesday,20.34,12201001 08:26:00,1291191960
2,536365,84406B,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,2010-12-01 08:26:00,morning,Wednesday,22.00,12201001 08:26:00,1291191960
3,536365,84029G,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01 08:26:00,morning,Wednesday,20.34,12201001 08:26:00,1291191960
4,536365,84029E,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01 08:26:00,morning,Wednesday,20.34,12201001 08:26:00,1291191960
...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,12,12/9/2011 12:50,0.85,12680.0,France,2011-12-09 12:50:00,afternoon,Friday,10.20,12201109 12:50:00,1323435000
541905,581587,22899,6,12/9/2011 12:50,2.10,12680.0,France,2011-12-09 12:50:00,afternoon,Friday,12.60,12201109 12:50:00,1323435000
541906,581587,23254,4,12/9/2011 12:50,4.15,12680.0,France,2011-12-09 12:50:00,afternoon,Friday,16.60,12201109 12:50:00,1323435000
541907,581587,23255,4,12/9/2011 12:50,4.15,12680.0,France,2011-12-09 12:50:00,afternoon,Friday,16.60,12201109 12:50:00,1323435000


## KPI Validation functions

### Validate that KPI & Datetime columns exists
**Preliminary Check: If this doesn't pass, other Validation checks will fail for this same reason.**

In [5]:
def column_exists(
    df: pd.core.frame.DataFrame,
    column_name: Union[str, List[str]]
) -> Tuple[bool, str]:
    """Validates if a column, or list of columns exists in the input DataFrame

    :param df: A pandas DataFrame
    :type df: pd.core.frame.DataFrame
    :param column_name: Name of the column to check existence. Or a list of column names to check existence of.
    :type column_name: Union[str, List[str]]
    :return: returns a tuple with the status as a bool and a status message
    :rtype: Tuple[bool, str]
    """
    # Lambda function templates to create string output
    valid_str = 'Accepted!'
    single_col_str = lambda col: f'"{col}" was not found as a column in the table!'
    multi_col_str = lambda cols: f'''{", ".join([f'"{col}"' for col in cols])} were not found as columns in the table!'''

    # column_name is a string. Single column to check
    if isinstance(column_name, str):
        status = column_name in df.columns
        message = valid_str if status else single_col_str(column_name)
        return status, message
    # column_name is a list. N cols to check
    else:
        # Find which cols do not exist in DataFrame
        not_found_cols = [col for col in column_name if col not in df.columns]
        
        if len(not_found_cols) == 1:
            return False, single_col_str(not_found_cols[0])
        if len(not_found_cols) > 1:
            return False, multi_col_str(not_found_cols)
        else:
            return True, valid_str

### Validate that aggregation type is valid for the column type (Categorical column won't work with mean/sum aggregation)
**Check #1 in documentation**

In [6]:
def validate_agg_type_fits_column(
    df: pd.core.frame.DataFrame, 
    column_name: str, 
    agg_type: str 
) -> Tuple[bool, str]:
    """Validates if agggregation type is supported and is valid for the specified column.

    :param df: A pandas DataFrame
    :type df: pd.core.frame.DataFrame
    :param column_name: Name of the column to validate
    :type column_name: str
    :param agg_type: A supported aggregation function
    :type agg_type: str
    :return: returns a tuple with the status as a bool and a status message
    :rtype: bool
    """
    # Check if aggregation type is supported
    SUPPORTED_AGGREGATIONS = ["mean", "sum", "count"]
    if agg_type not in SUPPORTED_AGGREGATIONS:
        return False, f'"{agg_type}" aggregation is not supported.\nSupported aggregations are {", ".join(SUPPORTED_AGGREGATIONS)}'

    # Check if trying to use a numerical aggregation on a categorical column.
    if str(df[column_name].dtype) == 'object' and agg_type != 'count':
        return False, f'"{column_name}" column is categorical. Quantitative data is required to perform {agg_type} aggregation.'

    return True, 'Accepted!'

### Validate that the KPI is not the date column.
**Check #2 in documentation**

In [7]:
def validate_kpi_not_datetime(
    df: pd.core.frame.DataFrame,
    kpi_column_name: str,
    date_column_name: str
) -> Tuple[bool, str]:
    """Validates if kpi column is not the same as the date column.

    :param df: A pandas DataFrame
    :type df: pd.core.frame.DataFrame
    :param kpi_column_name: Name of the column used for KPI
    :type kpi_column_name: str
    :param date_column_name: Name of the date column
    :type date_column_name: str
    :return: returns a tuple with the status as a bool and a status message
    :rtype: Tuple[bool, str]
    """
    status = kpi_column_name != date_column_name
    message = 'Accepted!' if status else 'KPI column cannot be the same as the date column'
    return status, message

### Validate that the date column is parseable
**Check #3 in documentation**

In [8]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime

def validate_date_column_is_parseable(
    df: pd.core.frame.DataFrame,
    date_column_name: str,
    date_format: str = None,
    unix_unit: str = None,
) -> Tuple[bool, str]:
    """Validates if specified date column is parseable.

    :param df: A pandas DataFrame
    :type df: pd.core.frame.DataFrame
    :param date_column_name: Name of the date column
    :type date_column_name: str
    :param date_format: Specified strftime to parse (https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior), defaults to None
    :type date_format: str, optional
    :param unix_unit: The time unit if specified date column is in Unix format, defaults to None
    :type unix_unit: str, optional
    :return: returns a tuple with the status as a bool and a status message
    :rtype: Tuple[bool, str]
    """
    valid_str = 'Accepted!'
    # Exit early if date column is a datetime object.
    if is_datetime(df[date_column_name]):
        return True, valid_str
    
    generic_err_msg = f'Unable to parse "{date_column_name}" column. Check that your date column is formatted properly and consistely.'
    out_of_bounds_msg = f'Timestamps in "{date_column_name}" were out of bounds. Check that your date column is formatted properly and consistely.'

    if unix_unit:
        # If a unix_unit is specified, it will try to convert with this unit
        try:
            pd.to_datetime(df[date_column_name], unit=unix_unit, infer_datetime_format=True)
        except pd.errors.OutOfBoundsDatetime as ex:
            return False, out_of_bounds_msg
        except Exception as ex:
            return False, f"{generic_err_msg}"
    elif date_format:
        # If a date_format is specified, it will try to convert with this format
        try:
            pd.to_datetime(df[date_column_name], format=date_format, infer_datetime_format=True)
        except pd.errors.OutOfBoundsDatetime as ex:
            return False, out_of_bounds_msg
        except Exception as ex:
            return False, f"{generic_err_msg}"
    else:
        # If neither date_format or unix_unit, let pandas do its best to infer datetime format.
        try:
            pd.to_datetime(df[date_column_name], infer_datetime_format=True)
        except pd.errors.OutOfBoundsDatetime as ex:
            return False, out_of_bounds_msg
        except Exception as ex:
            return False, f"{generic_err_msg}"
    
    # datetime column is parseable if code reaches here.
    return True, valid_str

### Wrapper function

In [9]:
def validate_kpi(
    df: pd.core.frame.DataFrame,
    kpi_column_name: str,
    agg_type: str,
    date_column_name: str,
    date_format: str = None,
    unix_unit: str = None,
    debug: bool = False,
) -> Tuple[bool, str]:
    """A wrapper function which cleanly invokes each validation check and breaks if it hits a falsy check.

    :param df: A pandas DataFrame
    :type df: pd.core.frame.DataFrame
    :param kpi_column_name: Name of the column used for KPI
    :type kpi_column_name: str
    :param agg_type: A supported aggregation function
    :type agg_type: str
    :param date_column_name: Name of the date column
    :type date_column_name: str
    :param date_format: Specified strftime to parse (https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior), defaults to None
    :type date_format: str, optional
    :param unix_unit: The time unit if specified date column is in Unix format, defaults to None
    :type unix_unit: str, optional
    :param debug: Bool for using debug mode with extra print statements at each validation, defaults to False
    :type debug: bool, optional
    :return: returns a tuple with the status as a bool and a status message
    :rtype: Tuple[bool, str]
    """

    # Preliminary Check that the KPI column exists
    # This check must be done independently. Otherwise, the other 3 checks will fail!
    status_bool, status_msg = column_exists(df, column_name=[kpi_column_name, date_column_name])
    if debug:
        print('Check #0: KPI column and Datetime column exist in DataFrame')
        print(status_bool, status_msg, sep=", ")
    if not status_bool:
        return status_bool, status_msg
    
    # Validation check results
    validations = [
        {'debug_str': 'Check #1: Validate column fits agg type', 'status': validate_agg_type_fits_column(df, column_name=kpi_column_name, agg_type=agg_type)}, 
        {'debug_str': 'Check #2: Validate kpi not datetime', 'status': validate_kpi_not_datetime(df, kpi_column_name=kpi_column_name, date_column_name=date_column_name)}, 
        {'debug_str': 'Check #3: Validate date column is parseable', 'status': validate_date_column_is_parseable(df, date_column_name=date_column_name, date_format=date_format, unix_unit=unix_unit)},
    ]
    for validation in validations:
        status_bool, status_msg = validation['status']
        if debug:
            print(validation['debug_str'])
            print(status_bool, status_msg, sep=", ")
        if not status_bool:
            return status_bool, status_msg

    # All Validation Checks have passed if code reaches here!
    return True, 'Accepted!'

## Test Cases

In [10]:
# Passes all checks
validate_kpi(ecom_df.tail(100), kpi_column_name='ItemTotalPrice', agg_type='sum', date_column_name='date')

(True, 'Accepted!')

In [11]:
# Passes all checks
# Example with debug mode
validate_kpi(ecom_df.tail(100), kpi_column_name='ItemTotalPrice', agg_type='sum', date_column_name='date', debug=True)

Check #0: KPI column and Datetime column exist in DataFrame
True, Accepted!
Check #1: Validate column fits agg type
True, Accepted!
Check #2: Validate kpi not datetime
True, Accepted!
Check #3: Validate date column is parseable
True, Accepted!


(True, 'Accepted!')

In [12]:
# Column does not exist
validate_kpi(ecom_df.tail(100), kpi_column_name='Price', agg_type='mean', date_column_name='date')

(False, '"Price" was not found as a column in the table!')

In [13]:
# Multiple columns do not exist
validate_kpi(ecom_df.tail(100), kpi_column_name='Price', agg_type='mean', date_column_name='day')

(False, '"Price", "day" were not found as columns in the table!')

In [14]:
# Should not be able to aggregate PeriodOfDay (categorical) with mean aggregation.
validate_kpi(ecom_df.tail(100), kpi_column_name='PeriodOfDay', agg_type='mean', date_column_name='date')

(False,
 '"PeriodOfDay" column is categorical. Quantitative data is required to perform mean aggregation.')

In [15]:
# kpi_column_name and date_column_name are the same. Should be Falsy
validate_kpi(ecom_df.tail(100), kpi_column_name='date', agg_type='mean', date_column_name='date')

(False, 'KPI column cannot be the same as the date column')

In [16]:
# kpi_column_name and date_column_name are the same. Should be Falsy
# Same as previous, but test with debug mode
validate_kpi(ecom_df.tail(100), kpi_column_name='date', agg_type='mean', date_column_name='date', debug=True)

Check #0: KPI column and Datetime column exist in DataFrame
True, Accepted!
Check #1: Validate column fits agg type
True, Accepted!
Check #2: Validate kpi not datetime
False, KPI column cannot be the same as the date column


(False, 'KPI column cannot be the same as the date column')

In [17]:
# Try running while inputting a custom date format
# date2 had following format (%m%Y%d %X)
# This should NOT pass, since I am using a different format.
validate_kpi(ecom_df.tail(100), kpi_column_name='ItemTotalPrice', agg_type='mean', date_column_name='date2', date_format="%Y%m%d%X")

(False,
 'Timestamps in "date2" were out of bounds. Check that your date column is formatted properly and consistely.')

In [18]:
# Try running while inputting a custom date format
# date2 had following format (%m%Y%d %X)
# This should pass as I'm using the same format
validate_kpi(ecom_df.tail(100), kpi_column_name='ItemTotalPrice', agg_type='mean', date_column_name='date2', date_format="%m%Y%d %X")

(True, 'Accepted!')

In [22]:
pd.to_datetime(ecom_df['unix_secs'], unit='ns')

0        1970-01-01 00:00:01.291191960
1        1970-01-01 00:00:01.291191960
2        1970-01-01 00:00:01.291191960
3        1970-01-01 00:00:01.291191960
4        1970-01-01 00:00:01.291191960
                      ...             
541904   1970-01-01 00:00:01.323435000
541905   1970-01-01 00:00:01.323435000
541906   1970-01-01 00:00:01.323435000
541907   1970-01-01 00:00:01.323435000
541908   1970-01-01 00:00:01.323435000
Name: unix_secs, Length: 531283, dtype: datetime64[ns]

In [20]:
validate_kpi(ecom_df.tail(100), kpi_column_name='ItemTotalPrice', agg_type='mean', date_column_name='unix_secs', unix_unit='ns')

(True, 'Accepted!')

In [82]:
from pandasql import sqldf

In [20]:
sql = '''
select dist_group, count(*)
from
(
 select case when UnitPrice between 0 and 10 then '(0, 10)'
             when UnitPrice between 10 and 20 then '(10, 20)' end as dist_group
 from ecom_df
)
group by dist_group
'''
sqldf(sql, locals())

,dist_group,count(*)
0,None,3767
1,"(0, 10)",507341
2,"(10, 20)",20175


In [21]:
sql = '''
SELECT Country, count() FROM ecom_df GROUP BY Country
'''
sqldf(sql, locals())

,Country,count()
0,Australia,1185
1,Austria,398
2,Bahrain,18
3,Belgium,2031
4,Brazil,32
5,Canada,151
6,Channel Islands,748
7,Cyprus,614
8,Czech Republic,25
9,Denmark,380
